In the past, querying a data object with a field name returned a NumPy `ndarray` . In the new unit system, data object queries will return a `YTArray`, a subclass of `ndarray` that preserves all of the nice properties of `ndarray`, including broadcasting, deep and shallow copies, and views. 

### Selecting data from an object

`YTArray` is 'unit-aware'.  Let's show how this works in practice using a sample Enzo dataset:

In [ ]:
import yt
ds = yt.load('IsolatedGalaxy/galaxy0030/galaxy0030')

dd = ds.all_data()
maxval, maxloc = ds.find_max('density')

dens = dd['density']

In [ ]:
print (maxval)

In [ ]:
print (dens)

In [ ]:
mass = dd['cell_mass']

print ("Cell Masses in CGS: \n", mass, "\n")
print ("Cell Masses in MKS: \n", mass.in_mks(), "\n")
print ("Cell Masses in Solar Masses: \n", mass.in_units('Msun'), "\n")
print ("Cell Masses in code units: \n", mass.in_units('code_mass'), "\n")

In [ ]:
dx = dd['dx']
print ("Cell dx in code units: \n", dx, "\n")
print ("Cell dx in centimeters: \n", dx.in_cgs(), "\n")
print ("Cell dx in meters: \n", dx.in_units('m'), "\n")
print ("Cell dx in megaparsecs: \n", dx.in_units('Mpc'), "\n")

### Unit conversions

YTArray defines several user-visible member functions that allow data to be converted from one unit system to another:

* `in_units`
* `in_cgs`
* `in_mks`
* `in_base`
* `convert_to_units`
* `convert_to_cgs`
* `convert_to_mks`
* `convert_to_base`

The first method, `in_units`, returns a copy of the array in the units denoted by a string argument:

In [ ]:
print (dd['density'].in_units('Msun/pc**3'))

`in_cgs` and `in_mks` return a copy of the array converted to CGS and MKS units, respectively:

In [ ]:
print (dd['pressure'])
print (dd['pressure'].in_cgs())
print (dd['pressure'].in_mks())

`in_cgs` and `in_mks` are just special cases of the more general `in_base`, which can convert a `YTArray` to a number of different unit systems:

In [ ]:
print (dd['pressure'].in_base('imperial')) # Imperial/English base units
print (dd['pressure'].in_base('galactic')) # Base units of kpc, Msun, Myr
print (dd['pressure'].in_base('planck')) # Base units in the Planck system
print (dd['pressure'].in_base()) # defaults to cgs if no argument given

`in_base` also takes the `"code"` argument to convert the `YTArray` into the base units of the dataset:

In [ ]:
print (dd['pressure'].in_base("code")) # The IsolatedGalaxy dataset from above

yt defines a number of unit systems, and new unit systems may be added by the user, which can also be passed to `in_base`. To learn more about the unit systems, how to use them with datasets and other objects, and how to add new ones, see [Unit Systems](unit_systems.html).

The rest of the methods do in-place conversions:

In [ ]:
dens = dd['density']
print (dens)

dens.convert_to_units('Msun/pc**3')
print (dens)

One possibly confusing wrinkle when using in-place conversions is if you try to query `dd['density']` again, you'll find that it has been converted to solar masses per cubic parsec:

In [ ]:
print (dd['density'])

dens.convert_to_units('g/cm**3')

print (dens)

Since the unit metadata is preserved and the array values are still correct in the new unit system, all numerical operations will still be correct.

One of the nicest aspects of this new unit system is that the symbolic algebra for mathematical operations on data with units is performed automatically by sympy.  This example shows how we can construct a field with density units from two other fields that have units of mass and volume:

In [ ]:
print (dd['cell_mass'])
print (dd['cell_volume'].in_units('cm**3'))

print ((dd['cell_mass']/dd['cell_volume']).in_cgs())

### Electrostatic/Electromagnetic Units

Electromagnetic units can be a bit tricky, because the units for such quantities in different unit systems can have entirely different dimensions, even if they are meant to represent the same physical quantities. For example, in the SI system of units, current in Amperes is a fundamental unit of measure, so the unit of charge "coulomb" is equal to one ampere-second. On the other hand, in the Gaussian/CGS system, there is no equivalent base electromagnetic unit, and the electrostatic charge unit "esu" is equal to one $\mathrm{cm^{3/2}g^{-1/2}s^{-1}}$ (which does not have any apparent physical significance). `yt` recognizes this difference:

In [ ]:
q1 = yt.YTArray(1.0,"C") # coulombs
q2 = yt.YTArray(1.0,"esu") # electrostatic units / statcoulomb

print ("units =", q1.in_mks().units, ", dims =", q1.units.dimensions)
print ("units =", q2.in_cgs().units, ", dims =", q2.units.dimensions)

In [ ]:
B1 = yt.YTArray(1.0,"T") # tesla
B2 = yt.YTArray(1.0,"gauss") # gauss

print ("units =", B1.in_mks().units, ", dims =", B1.units.dimensions)
print ("units =", B2.in_cgs().units, ", dims =", B2.units.dimensions)

To convert between these two systems, use [Unit Equivalencies](unit_equivalencies.html).

### Working with views and converting to ndarray

There are two ways to convert the data into a numpy array.  The most straightforward and safe way to do this is to create a copy of the array data.  The following cell demonstrates four equivalent ways of doing this, in increasing degree of terseness.

In [ ]:
import numpy as np

dens = dd['cell_mass']

print (dens.to_ndarray())
print (np.array(dens))
print (dens.value)
print (dens.v)

Since we have a copy of the data, we can mess with it however we wish without disturbing the original data returned by the yt data object.

There is yet another way to return a copy of the array data in a `YTArray` or the floating-point value of  a `YTQuantity`, which also allows for the possibility to convert to different units. This is done using the `to_value` method:

In [ ]:
print(dens.to_value()) # Don't change units
print(dens.to_value("Msun")) # Change units to solar masses
print(dens[0].to_value("lbm")) # Pick the first value and change its units to pounds

Another way to touch the raw array data is to get a _view_.  A numpy view is a lightweight array interface to a memory buffer. There are four ways to create views of YTArray instances:

In [ ]:
print (dd['cell_mass'].ndarray_view())
print (dd['cell_mass'].view(np.ndarray))
print (dd['cell_mass'].ndview)
print (dd['cell_mass'].d)

When working with views, remember that you are touching the raw array data and no longer have any of the unit checking provided by the unit system.  This can be useful where it might be more straightforward to treat the array as if it didn't have units but without copying the data.

In [ ]:
density_values = dd['density'].d
density_values[0:10] = 0

# The original array was updated
print (dd['density'])

### Round-Trip Conversions to and from Other Unit Systems

Finally, a `YTArray` or `YTQuantity` may be converted to an [AstroPy quantity](http://astropy.readthedocs.org/en/latest/units/), which is a NumPy array or a scalar associated with units from AstroPy's units system. You may use this facility if you have AstroPy installed. 

Some examples of converting from AstroPy units to yt:

In [ ]:
from astropy import units as u

x = 42.0 * u.meter
y = yt.YTQuantity.from_astropy(x)

In [ ]:
print (x, type(x))
print (y, type(y))

In [ ]:
a = np.random.random(size=10) * u.km/u.s
b = yt.YTArray.from_astropy(a)

In [ ]:
print (a, type(a))
print (b, type(b))

It also works the other way around, converting a `YTArray` or `YTQuantity` to an AstroPy quantity via the method `to_astropy`. For arrays:

In [ ]:
temp = dd["temperature"]
atemp = temp.to_astropy()

In [ ]:
print (temp, type(temp))
print (atemp, type(atemp))

and quantities:

In [ ]:
from yt.units import kboltz
kb = kboltz.to_astropy()

In [ ]:
print (kboltz, type(kboltz))
print (kb, type(kb))

As a sanity check, you can show that it works round-trip:

In [ ]:
k1 = kboltz.to_astropy()
k2 = yt.YTQuantity.from_astropy(kb)
print(k1)
print(k2)

In [ ]:
c = yt.YTArray.from_astropy(a)
d = c.to_astropy()
print (a == d)

We can also do the same thing with unitful quantities from the [Pint package](http://pint.readthedocs.org), using essentially the same procedure:

In [ ]:
from pint import UnitRegistry
ureg = UnitRegistry()
v = 1000.*ureg.km/ureg.s
w = yt.YTQuantity.from_pint(v)

In [ ]:
print (v, type(v))
print (w, type(w))

In [ ]:
ptemp = temp.to_pint()

In [ ]:
print (temp, type(temp))
print (ptemp, type(ptemp))

### Defining New Units

yt also provides a way to define your own units. Suppose you wanted to define a new unit for "miles per hour", the familiar "mph", which is not already in yt. One can do this by calling `yt.define_unit()`:

In [ ]:
yt.define_unit("mph", (1.0, "mile/hr"))

Once this unit is defined, it can be used in the same way as any other unit:

In [ ]:
from yt.units import clight
print (clight.to('mph'))

If you want to define a new unit which is prefixable (like SI units), you can set `prefixable=True` when defining the unit:

In [ ]:
from yt import YTQuantity
yt.define_unit("L", (1000.0, "cm**3"), prefixable=True)
print (YTQuantity(1.0, "mL").to("cm**3"))

`yt.define_unit()` defines new units for all yt operations. However, new units can be defined for particular datasets only as well using `ds.define_unit()`, which has the same signature:

In [ ]:
ds.define_unit("M_star", (2.0e13, "Msun"))

In [ ]:
dd = ds.all_data()
print(dd.quantities.total_mass().to("M_star"))